## 講座1_シンプルバージョンをベースにする

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pwd

'/content'

In [3]:
import os

import pandas as pd
import numpy as np
from glob import  glob

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
input_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/inputs/'
photo_dir = os.path.join(input_dir, 'photos')
photo_pathes = glob(os.path.join(photo_dir, "*.jpg"))
output_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/outputs/'

os.makedirs(output_dir, exist_ok=True)

train_df = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(input_dir, 'test.csv'))

material_df = pd.read_csv(os.path.join(input_dir, 'materials.csv'))
technique_df = pd.read_csv(os.path.join(input_dir, 'techniques.csv'))

In [5]:
import re
from requests import get

#!pip install ipynb-path
#import ipynb_path

class Config:
    N_FOLDS = 5
    N_EPOCHS = 50
    #NB_NAME = ''.join(re.findall('.*/(.*).ipynb', ipynb_path.get()))
    nb_name = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
    NB_NAME = re.findall('(.*).ipynb', nb_name)[0]

In [6]:
import torch
import random

def seed_torch(seed=1993):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [7]:
!pip uninstall -y scikit-learn
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

Found existing installation: scikit-learn 1.0.dev0
Uninstalling scikit-learn-1.0.dev0:
  Successfully uninstalled scikit-learn-1.0.dev0
Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple
     |████████████████████████████████| 22.9 MB 57.2 MB/s 


### 画像データの読み込み

In [8]:
from PIL import Image

def to_img_path(object_id):
    return os.path.join(photo_dir, f'{object_id}.jpg')

def read_image(object_id):
    return Image.open(to_img_path(object_id))

In [9]:
import torch
from torch import nn
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torch.utils import data

# torchvision
from torchvision import transforms as T
from torchvision.models import resnet34

# scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedGroupKFold

In [10]:
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    label_key = "target"
    
    @property
    def meta_keys(self):
        retbal = [self.object_path_key]
        
        if self.is_train:
            retbal += [self.label_key]
            
        return retbal
    
    def __init__(self, meta_df: pd.DataFrame, is_train=True):
        """
        args:
            meta_df: 
                画像へのパスと label 情報が含まれている dataframe
                必ず object_path に画像へのパス, target に正解ラベルが入っている必要があります
            
            is_train:
                True のとき学習用のデータ拡張を適用します.
                False の時は単に size にリサイズを行います
        """
        
        self.is_train = is_train
        for k in self.meta_keys:
            if k not in meta_df:
                raise ValueError("meta df muust have {}".format(k))
        
        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")
        
        size = (224, 224)
        
        additional_items = (
            [T.Resize(size)]
            # if not is_train
            # else [
            #     #T.RandomGrayscale(p=0.2),
            #     T.RandomVerticalFlip(),
            #     T.RandomHorizontalFlip(),
            #     T.ColorJitter(
            #         brightness=0.3,
            #         contrast=0.5,
            #         saturation=[0.8, 1.3],
            #         hue=[-0.05, 0.05],
            #     ),
            #     T.RandomResizedCrop(size),
            # ]
        )
        
        self.transformer = T.Compose(
            [*additional_items, T.ToTensor(), T.Normalize(mean=IMG_MEAN, std=IMG_STD)]
        )
        
    def __getitem__(self, index):
        data = self.index_to_data[index]
        obj_path, label = data.get(self.object_path_key), data.get(self.label_key, -1)
        img = Image.open(obj_path)
        img = self.transformer(img)
        return img, label
    
    def __len__(self):
        return len(self.meta_df)

In [11]:
assert torch.cuda.is_available()

DEVICE = torch.device("cuda")

## Train / Validation Phase

In [12]:
def train(
    model: nn.Module,
    optimizer: Optimizer,
    train_loader: data.DataLoader
) -> pd.Series:
    
    # train にすることで model 内の学習時にのみ有効な機構が有効になります (Dropouts Layers、BatchNorm Layers
    model.train()
    
    criterion = nn.MSELoss()
    
    for i, (x_i, y_i) in enumerate(train_loader):
        x_i = x_i.to(DEVICE)
        y_i = y_i.to(DEVICE).reshape(-1,1).float()
        
        output = model(x_i)
        loss = criterion(output, y_i)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

def predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:
    # train とは逆で model 内の学習時にのみ有効な機構がオフになります (Dropouts Layers、BatchNorm Layers...)
    model.eval()
    predicts = []
    
    for x_i, y_i in loader:
        
        # 明示的に勾配を計算しないように指定することができます. 
        # この関数ではモデルの更新はせずに単に出力だけを使いますので勾配は不要です.
        with torch.no_grad():
            output = model(x_i.to(DEVICE))
            
        predicts.extend(output.data.cpu().numpy())
        
    pred = np.array(predicts).reshape(-1)
    return pred

def calculate_metrics(y_true, y_pred) -> dict:
    """正解ラベルと予測ラベルから指標を計算する"""    
    return {
        'rmse': mean_squared_error(y_true, y_pred) ** .5
    }

def valid(
    model: nn.Module, 
    y_valid: np.ndarray, 
    valid_loader: data.DataLoader
) -> pd.Series:
    """検証フェーズ
    与えられたモデル・データローダを使って検証フェーズを実行。スコアの dict と予測した値を返す
    """
    
    pred = predict(model, valid_loader)
    print(f'y_valid={y_valid.shape} pred={pred.shape}')
    score = calculate_metrics(y_valid, pred)
    return score, pred
        

## Run Fold

1. train / valid の loader 作成
2. 以下を epoch 数だけ繰り返す
    1. 学習用データで学習 
    2. 検証用データで検証スコアの算出

In [13]:
def run_fold(
    model: nn.Module, 
    train_df: pd.DataFrame, 
    valid_df: pd.DataFrame, 
    y_valid: np.ndarray, 
    n_epochs=30) -> np.ndarray:
    """
    train / valid に分割されたデータで学習と同時に検証を行なう
    """
    
    # 0: 
    #   : 前準備. dataframe から data loader を作成
    train_dataset = AtmaDataset(meta_df=train_df)
    train_loader = data.DataLoader(
        train_dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=2
    )
    
    #   : 検証用の方は is_train=False にしてデータ拡張オフにする
    valid_dataset = AtmaDataset(meta_df=valid_df, is_train=False)
    valid_loader = data.DataLoader(valid_dataset, batch_size=256, num_workers=2)
    
    # optimizer の定義
    optimizer = Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(1, n_epochs + 1):
        print(f'start {epoch}')
        
        # 1: 学習用データで学習を実行。学習時のロスを取得
        train(model, optimizer, train_loader)
        
        # 2: 検証データでのスコアを計算
        score_valid, y_valid_pred = valid(model=model, valid_loader=valid_loader, y_valid=y_valid)
        
        print(score_valid)

### その他

モデル作成などの関数定義

In [14]:
pip install timm efficientnet_pytorch

In [15]:
import timm

# def create_model():
#     model = timm.create_model('efficientnet_b3', pretrained=False)
#     model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
#     return model

from efficientnet_pytorch import EfficientNet
def create_model():
    model = EfficientNet.from_name('efficientnet-b3')
    num_ftrs = model._fc.in_features
    model._fc = nn.Linear(in_features=num_ftrs, out_features=1, bias=True)
    return model

def create_metadata(input_df):
    out_df = input_df[['object_id']].copy()
    out_df['object_path'] = input_df['object_id'].map(to_img_path)
    
    if "target" in input_df:
        out_df["target"] = input_df["target"]
        
    return out_df

def run_test_predict(model):
    test_meta_df = create_metadata(test_df)
    
    test_dataset = AtmaDataset(meta_df=test_meta_df, is_train=False)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=256, drop_last=False, num_workers=2)
    
    y_pred = predict(model, loader=test_loader)
    return y_pred

In [16]:
!nvidia-smi

Wed Jul 21 12:24:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
train_meta_df = create_metadata(train_df)

test_predictions = []

fold = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=1993)
cv = list(fold.split(X= train_df, y=train_df["target"], groups=train_df["art_series_id"]))[:Config.N_FOLDS]

for i, (idx_tr, idx_valid) in enumerate(cv):
    print(f'==== start cv {i} ====')
    model = create_model()
    model.to(DEVICE)
    
    # 1. Fold の学習
    run_fold(
        model=model, 
        train_df=train_meta_df.iloc[idx_tr], 
        valid_df=train_meta_df.iloc[idx_valid], 
        y_valid=train_meta_df['target'].values[idx_valid],
        n_epochs=Config.N_EPOCHS
    )
    
    # 2. モデルで予測 (本当はローカルに保存した重みを読みだすなどするほうがあとで振り返りやすいが簡易にそのまま予測する)
    y_pred_i = run_test_predict(model)
    test_predictions.append(y_pred_i)
    del model

==== start cv 0 ====
start 1
y_valid=(775,) pred=(775,)
{'rmse': 1.2603324869370682}
start 2
y_valid=(775,) pred=(775,)
{'rmse': 0.9898516445502863}
start 3
y_valid=(775,) pred=(775,)
{'rmse': 0.9451204539720096}
start 4
y_valid=(775,) pred=(775,)
{'rmse': 0.9699566876789254}
start 5
y_valid=(775,) pred=(775,)
{'rmse': 1.042850114465067}
start 6
y_valid=(775,) pred=(775,)
{'rmse': 1.0562351710756461}
start 7
y_valid=(775,) pred=(775,)
{'rmse': 0.9501718158305824}
start 8
y_valid=(775,) pred=(775,)
{'rmse': 0.9298075109308624}
start 9
y_valid=(775,) pred=(775,)
{'rmse': 1.0723352113153921}
start 10
y_valid=(775,) pred=(775,)
{'rmse': 1.0056347166145825}
start 11
y_valid=(775,) pred=(775,)
{'rmse': 1.0517239463910535}
start 12
y_valid=(775,) pred=(775,)
{'rmse': 0.9004626171673926}
start 13
y_valid=(775,) pred=(775,)
{'rmse': 0.9151002321225321}
start 14
y_valid=(775,) pred=(775,)
{'rmse': 0.9100562437583714}
start 15
y_valid=(775,) pred=(775,)
{'rmse': 0.9336203162019915}
start 16
y_val

In [18]:
# すべての予測の平均値を使う
pred_mean = np.array(test_predictions).mean(axis=0)

pd.DataFrame({
    "target": pred_mean
}).to_csv(os.path.join(output_dir, Config.NB_NAME + ".csv"), index=False)